In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, average_precision_score)

Чтобы обеспечить прозрачность, контроль и воспроизводимость всей цепочки подготовки данных к моделированию, применяем **все ключевые шаги очистки и преобразования** и создаем датафрейм `df_model`, для использования во всех экспериментах.

In [ ]:
file_id = '1ATASmLwbd-sVPOJ7kChU9fumc15lRvwK'
download_url = f'https://drive.google.com/uc?id={file_id}&export=download'
df = pd.read_csv(download_url)

In [ ]:
# 1. Удаляем пропуски в Income
df = df.dropna(subset=['Income'])

# 2. Удаляем аномалии / выбросы
df = df.dropna(subset=['Income']).copy()
df = df[(~df['Year_Birth'].isin([1893, 1899, 1900])) &
        (df['Income'] != 666_666)].copy()

# 3. Чистим категорию Marital_Status: «Absurd», «YOLO», «Alone» трактуем как «Single»
df.loc[:, 'Marital_Status'] = (df['Marital_Status'].replace({'Absurd': 'Single', 'YOLO': 'Single', 'Alone': 'Single'}))

# 4. Feature engineering
#    4.1. Total_Spending — сумма трат за всё время
spend_cols = ['MntWines', 'MntFruits', 'MntMeatProducts',
              'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']
df['Total_Spending'] = df[spend_cols].sum(axis=1)

# 5. One-Hot Encoding категорий
df = pd.get_dummies(df,
                    columns=['Education', 'Marital_Status'],
                    drop_first=True,
                    dtype=int)

# 6. Удаляем неинформативные столбцы
df = df.drop(columns=['Id', 'Dt_Customer'], errors='ignore')

print("Форма после очистки:", df.shape)
print("Общее NaN:", df.isna().sum().sum())
assert set(df['Response'].unique()) <= {0, 1}, "Response должен быть 0/1"

df_model = df.copy()   # датафрейм для ML

Форма после очистки: (2212, 27)
Общее NaN: 0


In [ ]:
df_model.info(
    verbose=True,
    show_counts=True
)

<class 'pandas.core.frame.DataFrame'>
Index: 2212 entries, 0 to 2239
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Year_Birth               2212 non-null   int64  
 1   Income                   2212 non-null   float64
 2   Kidhome                  2212 non-null   int64  
 3   Teenhome                 2212 non-null   int64  
 4   Recency                  2212 non-null   int64  
 5   MntWines                 2212 non-null   int64  
 6   MntFruits                2212 non-null   int64  
 7   MntMeatProducts          2212 non-null   int64  
 8   MntFishProducts          2212 non-null   int64  
 9   MntSweetProducts         2212 non-null   int64  
 10  MntGoldProds             2212 non-null   int64  
 11  NumDealsPurchases        2212 non-null   int64  
 12  NumWebPurchases          2212 non-null   int64  
 13  NumCatalogPurchases      2212 non-null   int64  
 14  NumStorePurchases        2212

## 1 Формирование выборок

Разделите данные на обучающую и тестовую и, при необходимости, валидационную выборки.

In [ ]:
# 1. train/test  = 80/20  (stratified)
X = df_model.drop('Response', axis=1)
y = df_model['Response']

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.20, stratify=y, random_state=42)

# 2. Объект CV для всех дальнейших экспериментов
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

## 2 Модель логистической регрессии


In [ ]:
num_cols = [c for c in X_train.columns
            if not c.startswith(('Education_', 'Marital_Status_'))]

numeric_pipe = Pipeline([('imputer', SimpleImputer(strategy='median')),
                         ('scaler' , StandardScaler())])

preproc = ColumnTransformer([('num', numeric_pipe, num_cols)], remainder='passthrough')

logreg = LogisticRegression(class_weight='balanced',
                            solver='liblinear',
                            max_iter=2000,
                            random_state=42)

pipe = Pipeline([('prep', preproc),
                 ('clf' , logreg)])

# подбор penalty и C
param_grid = {'clf__penalty'      : ['l1', 'l2'],
              'clf__C'            : np.logspace(-3, 2, 10),
              'clf__class_weight' : [None, 'balanced']}


search = GridSearchCV(pipe,
                      param_grid=param_grid,
                      scoring='roc_auc',
                      cv=cv,
                      n_jobs=-1,
                      verbose=1)

search.fit(X_train, y_train)

print("Лучшие параметры:", search.best_params_)
print("Средний ROC AUC (CV):", search.best_score_.round(4))

best_model = search.best_estimator_     # уже переобучен внутри .fit

# финальные метрики на test
y_pred  = best_model.predict(X_test)
y_prob  = best_model.predict_proba(X_test)[:, 1]

metrics_test = {
    'Accuracy' : accuracy_score(y_test, y_pred),
    'Precision': precision_score(y_test, y_pred, zero_division=0),
    'Recall'   : recall_score(y_test, y_pred),
    'F1-score' : f1_score(y_test, y_pred),
    'ROC AUC'  : roc_auc_score(y_test, y_prob),
    'PR AUC'   : average_precision_score(y_test, y_prob)
}

print("\n Метрики на test")
for k, v in metrics_test.items():
    print(f"{k:<9}: {v:.4f}")

# значимые признаки
# порядок фичей после ColumnTransformer: сначала scaled num, затем passthrough dummies
feature_names = num_cols + [c for c in X_train.columns if c not in num_cols]
coefs = best_model.named_steps['clf'].coef_[0]
top10 = (pd.Series(coefs, index=feature_names)
           .sort_values(key=np.abs, ascending=False)
           .head(10))
print("\nTOP-10 признаков по |beta|:")
print(top10)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
Лучшие параметры: {'clf__C': np.float64(0.1668100537200059), 'clf__class_weight': 'balanced', 'clf__penalty': 'l1'}
Средний ROC AUC (CV): 0.8329

 Метрики на test
Accuracy : 0.7946
Precision: 0.4048
Recall   : 0.7612
F1-score : 0.5285
ROC AUC  : 0.8703
PR AUC   : 0.5731

TOP-10 признаков по |beta|:
Marital_Status_Together   -1.095024
Marital_Status_Married    -0.915329
Recency                   -0.683902
NumWebVisitsMonth          0.681884
Teenhome                  -0.632618
NumStorePurchases         -0.628542
Education_PhD              0.488768
NumCatalogPurchases        0.475652
Income                     0.424295
MntWines                   0.307744
dtype: float64


### Логистическая регрессия — итоговые результаты (GridSearch с `class_weight ∈ {None, balanced}`)
Была построена модель логистической регрессии для предсказания отклика клиента (Response).
- Предобработка: Числовые признаки были стандартизированы (StandardScaler), пропуски заполнены медианным значением.
- Борьба с дисбалансом: Был использован параметр class_weight='balanced'
- Регуляризация и отбор признаков: С помощью GridSearchCV была выбрана L1-регуляризация (penalty='l1'), которая борется с переобучением и автоматически обнуляет коэффициенты у наименее важных признаков, выполняя их отбор.

| Метрика (test-set, 443 строк) | Значение |
|-------------------------------|----------|
| **Accuracy**                  | **0.795** |
| **Precision**                 | **0.405** |
| **Recall**                    | **0.761** |
| **F1-score**                  | **0.529** |
| **ROC AUC**                   | **0.870** |
| **PR AUC**                    | **0.573** |

*Лучшие гиперпараметры (5-fold CV, ROC AUC = 0.833):*  
`penalty = 'l1'`, `C ≈ 0.17`, `class_weight = 'balanced'`

**1. Оценка качества модели**

Финальная модель, обученная на лучших параметрах (C=0.167, penalty='l1', class_weight='balanced'), показала высокое предсказательное качество на тестовой выборке:
* ROC AUC = 0.870: модель хорошо умеет ранжировать клиентов, которые с большей вероятностью откликнутся, выше тех, кто не откликнется.
* PR AUC = 0.573  подтверждает высокое качество, особенно в условиях дисбаланса классов.

Анализ метрик, зависящих от порога отсечения (0.5), показывает интересный компромисс:
* Recall = 0.761: Модель успешно находит 76% всех клиентов, которые действительно откликнулись на предложение. Это сильная сторона модели.
* Precision = 0.405: Когда модель предсказывает отклик, она оказывается права только в 40.5% случаев. Это означает, что среди всех, кого модель пометила как "откликнувшихся", почти 60% на самом деле не откликнулись (False Positives).

**Вывод по метрикам**: Модель настроена на максимизацию охвата целевой аудитории (высокий Recall) ценой большого количества ложноположительных срабатываний (низкий Precision). Это оправдано, если стоимость рассылки предложения низкая.

---

#### TOP-10 признаков, оказавших наибольшее влияние на результат:

| № | Признак | beta-коэффициент |
|---|---------|--------------|
| 1 | `Marital_Status_Together` | −1.10 |
| 2 | `Marital_Status_Married`  | −0.92 |
| 3 | `Recency`                 | −0.68 |
| 4 | `NumWebVisitsMonth`       | +0.68 |
| 5 | `Teenhome`                | −0.63 |
| 6 | `NumStorePurchases`       | −0.63 |
| 7 | `Education_PhD`           | +0.49 |
| 8 | `NumCatalogPurchases`     | +0.48 |
| 9 | `Income`                  | +0.42 |
| 10| `MntWines`                | +0.31 |

Благодаря L1-регуляризации можно интерпретировать коэффициенты (beta) как меру влияния признака на вероятность отклика.

**Ключевые инсайты:**

* Семейное положение — самый сильный предиктор. Клиенты, состоящие в браке или живущие вместе, значительно реже откликаются на предложение.
* Поведенческие паттерны важнее демографии. Признаки, описывающие покупательское поведение (Recency, NumWebVisitsMonth, NumStorePurchases), имеют очень высокие по модулю коэффициенты.

**Портрет "идеального" клиента.** Наиболее вероятно откликнется клиент, который:
1. Не состоит в браке/отношениях.
2. Давно не совершал покупок.
3. Часто заходит на сайт, но редко покупает в физических магазинах.
4. Имеет высокий доход и степень PhD.
5. Активно покупает по каталогам.
6. Не имеет подростков дома.

---

##### Выводы
* **Балансировка весов (`class_weight='balanced'`) выбрана поиском** — при `C ≈ 0.17` даёт наивысший ROC AUC.  
* Модель отделяет потенциальных покупателей (*ROC AUC ≈ 0.87*) и удерживает высокий **Recall ≈ 0.76**, сохраняя приемлемую Precision.  
* Самыми «отрицательными» факторами остаются семейные статусы *Together* и *Married*, а также большая давность последней покупки (`Recency`).  
* Повышают вероятность отклика высокая степень (`Education_PhD`), повышенная онлайн-активность (`NumWebVisitsMonth`) и более высокие доходы.


## 3 Экспиременты для улучшения качества.

### Эксперимент 1: Feature Engineering

Цель: Проверить, даст ли статистически значимый признак IsMultichannel и другие сгенерированные признаки прирост в качестве комплексной модели.

1. Создать новые признаки:

На основе гипотезы 4: IsMultichannel и ChannelCount.

Другие осмысленные признаки:

Customer_Tenure (срок жизни клиента с момента регистрации).

TotalMnt (общие траты на все категории).

TotalPurchases (общее число покупок по всем каналам).

IsParent (бинарный флаг, есть ли дети: Kidhome + Teenhome > 0).

AvgPurchaseValue (средний чек: TotalMnt / TotalPurchases).

2. Переобучить исходный пайплайн с LogisticRegression на расширенном наборе данных (не забудьте добавить новые числовые признаки в num_cols).

Зафиксировать новый ROC AUC на кросс-валидации и сравнить с baseline.

In [ ]:
import numpy as np, pandas as pd, warnings
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (roc_auc_score, average_precision_score,
                             accuracy_score, precision_score, recall_score, f1_score)

warnings.filterwarnings('ignore')

# df_fe для исследования и создания новых признаков
df_fe = df_model.copy()

# Новые признаки
df_fe['ChannelCount']   = ((df_fe['NumStorePurchases']   > 0).astype(int) +
                           (df_fe['NumWebPurchases']     > 0).astype(int) +
                           (df_fe['NumCatalogPurchases'] > 0).astype(int))
df_fe['IsMultichannel'] = (df_fe['ChannelCount'] == 3).astype(int)


df_fe['TotalPurchases'] = (df_fe['NumStorePurchases'] +
                           df_fe['NumWebPurchases']   +
                           df_fe['NumCatalogPurchases'])
df_fe['TotalMnt'] = (df_fe['MntWines'] + df_fe['MntFruits'] + df_fe['MntMeatProducts'] +
                     df_fe['MntFishProducts'] + df_fe['MntSweetProducts'] + df_fe['MntGoldProds'])


df_fe['IsParent'] = ((df_fe['Kidhome'] + df_fe['Teenhome']) > 0).astype(int)


df_fe['AvgPurchaseValue'] = np.where(df_fe['TotalPurchases'] > 0,
                                     df_fe['TotalMnt'] / df_fe['TotalPurchases'],
                                     0)

# Разделение на выборки
X = df_fe.drop('Response', axis=1)
y = df_fe['Response']
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.20,
                                          stratify=y, random_state=42)

num_cols = X_tr.select_dtypes(include=np.number).columns.tolist()

# Pipeline + GridSearch
numeric = Pipeline([('imp', SimpleImputer(strategy='median')),
                    ('sc' , StandardScaler())])
prep = ColumnTransformer([('num', numeric, num_cols)],
                         remainder='passthrough')

logreg = LogisticRegression(max_iter=2000, solver='liblinear', random_state=42)
pipe   = Pipeline([('prep', prep), ('clf', logreg)])

param_grid = {
    'clf__penalty'     : ['l1','l2'],
    'clf__C'           : np.logspace(-3, 2, 8),
    'clf__class_weight': [None, 'balanced']
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
search = GridSearchCV(pipe, param_grid, scoring='roc_auc',
                      cv=cv, n_jobs=-1, verbose=1)
search.fit(X_tr, y_tr)

print("\nЛучшие гиперпараметры:", search.best_params_,
      "\nCV ROC-AUC  :", round(search.best_score_,4))

best = search.best_estimator_
y_pred = best.predict(X_te)
y_prob = best.predict_proba(X_te)[:,1]

metrics = {
    'Accuracy' : accuracy_score(y_te, y_pred),
    'Precision': precision_score(y_te, y_pred, zero_division=0),
    'Recall'   : recall_score(y_te, y_pred),
    'F1-score' : f1_score(y_te, y_pred),
    'ROC AUC'  : roc_auc_score(y_te, y_prob),
    'PR AUC'   : average_precision_score(y_te, y_prob)
}
print("\nTest-метрики")
for k,v in metrics.items():
    print(f"{k:<9}: {v:.4f}")

Fitting 5 folds for each of 32 candidates, totalling 160 fits

Лучшие гиперпараметры: {'clf__C': np.float64(3.7275937203149416), 'clf__class_weight': 'balanced', 'clf__penalty': 'l1'} 
CV ROC-AUC  : 0.8491

Test-метрики
Accuracy : 0.7878
Precision: 0.3937
Recall   : 0.7463
F1-score : 0.5155
ROC AUC  : 0.8817
PR AUC   : 0.6632


В ходе подбора гиперпараметров было обнаружено два близких по ROC AUC решения. Первое (class_weight=None) обеспечивает высокую точность (Precision=0.86), но крайне низкий охват (Recall=0.37), находя лишь треть целевой аудитории. Второе решение (class_weight='balanced') обеспечивает значительно более высокий охват (Recall=0.75) при приемлемой точности. Учитывая бизнес-задачу по максимизации отклика на маркетинговую кампанию, где стоимость ложноположительного срабатывания низка, модель с высоким Recall является более предпочтительной. Поэтому для дальнейших экспериментов и финальной оценки выбрана модель с параметрами: C≈3.73, penalty='l1', class_weight='balanced'.


### Эксперимент 2: Feature Selection (Отбор признаков)

Цель: убрать взаимно-избыточные фичи и посмотреть, выиграет ли от этого логрега (стабильность/качество).

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import (roc_auc_score, average_precision_score,
                             accuracy_score, precision_score,
                             recall_score, f1_score)
import numpy as np, pandas as pd

# Используем данные после Эксперимента 1 (Feature Engineering)
# 2.1  Корреляционный фильтр
num_cols_all = X_tr.select_dtypes(include=np.number).columns
corr_matrix = X_tr[num_cols_all].corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]

X_tr_fs = X_tr.drop(columns=to_drop)
X_te_fs = X_te.drop(columns=to_drop)

print(f"После корреляционного фильтра осталось {X_tr_fs.shape[1]} признаков (удалено {len(to_drop)}: {to_drop})")


# 2.2  Предобработка данных перед RFECV
# числовые и категориальные колонки в отфильтрованных данных
num_cols_fs = X_tr_fs.select_dtypes(include=np.number).columns.tolist()
cat_cols_fs = X_tr_fs.select_dtypes(exclude=np.number).columns.tolist()

preprocessor = ColumnTransformer(
    [('num', Pipeline([('imp', SimpleImputer(strategy='median')), ('sc', StandardScaler())]), num_cols_fs)],
    remainder='passthrough'
)

X_tr_prepared = preprocessor.fit_transform(X_tr_fs)
X_te_prepared = preprocessor.transform(X_te_fs)

# имена всех колонок после трансформации
feature_names_prepared = preprocessor.get_feature_names_out()


#  2.3  RFECV на подготовленных данных
# Базовый классификатор для RFECV
base_clf = LogisticRegression(
    C=3.73, penalty='l1', class_weight='balanced', # Гиперпараметры из эксп. 1
    max_iter=2000, solver='liblinear', random_state=42
)

rfecv = RFECV(
    estimator=base_clf,
    step=1,
    min_features_to_select=10,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring='roc_auc',
    n_jobs=-1
)

# Обучение RFECV на подготовленных данных
rfecv.fit(X_tr_prepared, y_tr)

# Отбор колонок в подготовленных данных
X_tr_sel = X_tr_prepared[:, rfecv.support_]
X_te_sel = X_te_prepared[:, rfecv.support_]

# Имена отобранных признаков
selected_feature_names = feature_names_prepared[rfecv.support_]
print(f"RFECV отобрал {len(selected_feature_names)} признаков.")


# 2.4  Финальный GridSearch на отобранных признаках
# т.к. данные уже обработаны, пайплайн не нужен
logreg_final = LogisticRegression(max_iter=2000, solver='liblinear', random_state=42)

param_grid = {
    'penalty': ['l1', 'l2'],
    'C': np.logspace(-3, 2, 8),
    'class_weight': [None, 'balanced']
}

gs = GridSearchCV(
    estimator=logreg_final,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    n_jobs=-1,
    verbose=1
)

gs.fit(X_tr_sel, y_tr)

print("\nЛучшие гиперпараметры:", gs.best_params_,
      "\nCV ROC-AUC  :", round(gs.best_score_, 4))

best2 = gs.best_estimator_
y_pred = best2.predict(X_te_sel)
y_prob = best2.predict_proba(X_te_sel)[:, 1]

metrics2 = {
    'Accuracy': accuracy_score(y_te, y_pred),
    'Precision': precision_score(y_te, y_pred, zero_division=0),
    'Recall': recall_score(y_te, y_pred),
    'F1-score': f1_score(y_te, y_pred),
    'ROC AUC': roc_auc_score(y_te, y_prob),
    'PR AUC': average_precision_score(y_te, y_prob)
}
print("\nTest-метрики после Feature Selection")
for k, v in metrics2.items():
    print(f"{k:<9}: {v:.4f}")

После корреляционного фильтра осталось 29 признаков (удалено 3: ['IsMultichannel', 'TotalMnt', 'AvgPurchaseValue'])
RFECV отобрал 23 признаков.
Fitting 5 folds for each of 32 candidates, totalling 160 fits

Лучшие гиперпараметры: {'C': np.float64(0.02682695795279726), 'class_weight': 'balanced', 'penalty': 'l2'} 
CV ROC-AUC  : 0.8479

Test-метрики после Feature Selection
Accuracy : 0.7878
Precision: 0.4029
Recall   : 0.8358
F1-score : 0.5437
ROC AUC  : 0.8891
PR AUC   : 0.6549


### Эксперимент 3: Gradient Boosting

Цель - убедиться, способен ли нелинейный ансамбль превзойти лучшую логестическую регрессию (Exp-2: ROC AUC = 0.889 / PR AUC = 0.655).


#### HistGradientBoosting

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, StratifiedKFold
from scipy.stats import loguniform, randint
from sklearn.metrics import (roc_auc_score, average_precision_score,
                             accuracy_score, precision_score,
                             recall_score, f1_score)

# Данные из Эксперимента 2 (NumPy-массивы)
# Шаг 1: Грубый поиск с помощью RandomizedSearchCV

base_hgb = HistGradientBoostingClassifier(
    loss='log_loss',
    class_weight='balanced',
    random_state=42
)

param_dist = {
    'learning_rate'    : loguniform(0.01, 0.3),
    'max_depth'        : randint(3, 10),
    'max_leaf_nodes'   : randint(20, 60),
    'min_samples_leaf' : randint(20, 100),
    'l2_regularization': loguniform(1e-3, 10),
    'max_iter'         : randint(100, 500)
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

rnd_search = RandomizedSearchCV(
    estimator=base_hgb,
    param_distributions=param_dist,
    n_iter=50,
    scoring='roc_auc',
    cv=cv,
    n_jobs=-1,
    random_state=42,
    verbose=1
)

rnd_search.fit(X_tr_sel, y_tr)

print("\nРезультаты RandomizedSearch")
print("Лучшие параметры (грубо):", rnd_search.best_params_)
print("CV ROC-AUC:", round(rnd_search.best_score_, 4))


# Шаг 2: Точный поиск с помощью GridSearchCV
best_params_rnd = rnd_search.best_params_

param_grid = {
    'learning_rate'    : [best_params_rnd['learning_rate'] * 0.8, best_params_rnd['learning_rate'], best_params_rnd['learning_rate'] * 1.2],
    'max_depth'        : [best_params_rnd['max_depth'] - 1, best_params_rnd['max_depth'], best_params_rnd['max_depth'] + 1],
    'max_leaf_nodes'   : [best_params_rnd['max_leaf_nodes'] - 10, best_params_rnd['max_leaf_nodes'], best_params_rnd['max_leaf_nodes'] + 10],
    'min_samples_leaf' : [max(1, best_params_rnd['min_samples_leaf'] - 10), best_params_rnd['min_samples_leaf'], best_params_rnd['min_samples_leaf'] + 10],
    'l2_regularization': [best_params_rnd['l2_regularization'] * 0.5, best_params_rnd['l2_regularization'], best_params_rnd['l2_regularization'] * 2],
    'max_iter'         : [best_params_rnd['max_iter']]
}

# Используем тот же базовый классификатор
grid_search = GridSearchCV(
    estimator=base_hgb,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=cv,
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_tr_sel, y_tr)

print("\nРезультаты GridSearchCV")
print("Лучшие финальные параметры:", grid_search.best_params_)
print("Финальный CV ROC-AUC:", round(grid_search.best_score_, 4))


# Шаг 3: Финальная оценка лучшей модели
best_model_gb = grid_search.best_estimator_

y_pred = best_model_gb.predict(X_te_sel)
y_prob = best_model_gb.predict_proba(X_te_sel)[:, 1]

metrics3 = {
    'Accuracy': accuracy_score(y_te, y_pred),
    'Precision': precision_score(y_te, y_pred, zero_division=0),
    'Recall': recall_score(y_te, y_pred),
    'F1-score': f1_score(y_te, y_pred),
    'ROC AUC': roc_auc_score(y_te, y_prob),
    'PR AUC': average_precision_score(y_te, y_prob)
}

print("\nTest-метрики после Gradient Boosting (GridSearch)")
for k, v in metrics3.items():
    print(f"{k:<9}: {v:.4f}")

Fitting 5 folds for each of 50 candidates, totalling 250 fits

Результаты RandomizedSearch
Лучшие параметры (грубо): {'l2_regularization': np.float64(5.1674258133224145), 'learning_rate': np.float64(0.042902233912047165), 'max_depth': 7, 'max_iter': 194, 'max_leaf_nodes': 54, 'min_samples_leaf': 79}
CV ROC-AUC: 0.8647
Fitting 5 folds for each of 243 candidates, totalling 1215 fits

Результаты GridSearchCV
Лучшие финальные параметры: {'l2_regularization': np.float64(2.5837129066612072), 'learning_rate': np.float64(0.0514826806944566), 'max_depth': 8, 'max_iter': 194, 'max_leaf_nodes': 44, 'min_samples_leaf': 69}
Финальный CV ROC-AUC: 0.8664

Test-метрики после Gradient Boosting (GridSearch)
Accuracy : 0.8600
Precision: 0.5263
Recall   : 0.7463
F1-score : 0.6173
ROC AUC  : 0.9033
PR AUC   : 0.6761


#### Random Forest (RF)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.metrics import roc_auc_score, average_precision_score, recall_score, precision_score

# Данные из эксперимента 2
rf_clf = RandomForestClassifier(
    class_weight='balanced',
    n_jobs=-1,
    random_state=42
)

rf_param = {
    'n_estimators'      : randint(200, 800),
    'max_depth'         : randint(5, 25),
    'min_samples_split' : randint(2, 15),
    'min_samples_leaf'  : randint(1, 10),
    'max_features'      : ['sqrt', 'log2', None]
}

rf_search = RandomizedSearchCV(
    estimator=rf_clf,
    param_distributions=rf_param,
    n_iter=40,
    scoring='roc_auc',
    cv=cv, # cv из предыдущего шага
    n_jobs=-1,
    random_state=42,
    verbose=1
)
rf_search.fit(X_tr_sel, y_tr)

# Оценка лучшей модели RF
rf_best = rf_search.best_estimator_
rf_cv_auc = rf_search.best_score_
rf_pred = rf_best.predict(X_te_sel)
rf_prob = rf_best.predict_proba(X_te_sel)[:, 1]

rf_metrics = {
    'ROC AUC': roc_auc_score(y_te, rf_prob),
    'PR AUC': average_precision_score(y_te, rf_prob),
    'Recall': recall_score(y_te, rf_pred),
    'Precision': precision_score(y_te, rf_pred, zero_division=0)
}
print("\nРезультаты RandomForest")
print(f"Лучшие параметры: {rf_search.best_params_}")
print(f"RF CV-AUC: {rf_cv_auc:.4f}, Test ROC-AUC: {rf_metrics['ROC AUC']:.4f}")

Fitting 5 folds for each of 40 candidates, totalling 200 fits

Результаты RandomForest
Лучшие параметры: {'max_depth': 21, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 9, 'n_estimators': 280}
RF CV-AUC: 0.8647, Test ROC-AUC: 0.9004


#### LightGBM (LGBMClassifier)

In [ ]:
from lightgbm import LGBMClassifier
from scipy.stats import loguniform

# Данные из эксперимента 2
scale_pos_weight = (len(y_tr) - y_tr.sum()) / y_tr.sum()

lgb_clf = LGBMClassifier(
    objective='binary',
    boosting_type='gbdt',
    n_jobs=-1,
    random_state=42,
    scale_pos_weight=scale_pos_weight
)

lgb_dist = {
    'learning_rate': loguniform(0.01, 0.2),
    'num_leaves'   : randint(15, 64),
    'n_estimators' : randint(300, 900),
    'max_depth'    : randint(3, 9),
    'reg_alpha'    : loguniform(1e-4, 1),
    'reg_lambda'   : loguniform(1e-4, 1)
}

lgb_rnd = RandomizedSearchCV(
    estimator=lgb_clf,
    param_distributions=lgb_dist,
    n_iter=40,
    scoring='roc_auc',
    cv=cv, # cv из предыдущего шага
    n_jobs=-1,
    random_state=42,
    verbose=1
)
lgb_rnd.fit(X_tr_sel, y_tr)

# Оценка лучшей модели LGBM
lgb_best = lgb_rnd.best_estimator_
lgb_cv_auc = lgb_rnd.best_score_
lgb_pred = lgb_best.predict(X_te_sel)
lgb_prob = lgb_best.predict_proba(X_te_sel)[:, 1]

lgb_metrics = {
    'ROC AUC': roc_auc_score(y_te, lgb_prob),
    'PR AUC': average_precision_score(y_te, lgb_prob),
    'Recall': recall_score(y_te, lgb_pred),
    'Precision': precision_score(y_te, lgb_pred, zero_division=0)
}
print("\n Результаты LightGBM")
print(f"Лучшие параметры: {lgb_rnd.best_params_}")
print(f"LGBM CV-AUC: {lgb_cv_auc:.4f}, Test ROC-AUC: {lgb_metrics['ROC AUC']:.4f}")

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[LightGBM] [Info] Number of positive: 266, number of negative: 1503
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000250 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1430
[LightGBM] [Info] Number of data points in the train set: 1769, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.150367 -> initscore=-1.731722
[LightGBM] [Info] Start training from score -1.731722
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM

In [ ]:
y_pred_lgb = lgb_best.predict(X_te_sel)
y_prob_lgb = lgb_best.predict_proba(X_te_sel)[:,1]

print("PR AUC :", average_precision_score(y_te, y_prob_lgb))
print("Recall :", recall_score(y_te, y_pred_lgb))
print("Precision:", precision_score(y_te, y_pred_lgb, zero_division=0))
print("F1-score :", f1_score(y_te, y_pred_lgb))

PR AUC : 0.684088956555306
Recall : 0.7761194029850746
Precision: 0.49056603773584906
F1-score : 0.6011560693641619


## 4 Описание экспериментов.

Какого качества получилось достичь? Что дало наибольший прирост качества?

### 1. Сводная таблица метрик

| Модель                       | ROC AUC (CV) | ROC AUC (test) | PR AUC | Recall | Precision | Изменения / Гиперпараметры                                |
|-----------------------------|--------------|----------------|--------|--------|-----------|----------------------------------------------------|
| **0. LogReg**               | 0.833        | 0.870          | 0.573  | 0.731  | 0.408     | L1-пен., `class_weight`                           |
| **1. + Feat.Eng.**          | 0.849        | 0.882          | 0.663  | 0.746  | 0.394     | +`IsMultichannel`, `TotalMnt`, `IsParent` и др.   |
| **2. + Feat.Sel.**          | 0.848        | 0.889          | 0.655  | 0.836  | 0.403     | Corr-filter (`p<0.90`)                             |
| **3а. HistGB**              | 0.866        | 0.903          | 0.676  | 0.746  | 0.526     | `lr=0.05`, `depth=8`, `balanced`                  |
| **3b. RF**                  | 0.865        | 0.900          | 0.670  | 0.746  | 0.530     | 280 деревьев, `depth=21`, `balanced`              |
| **3c. LightGBM (лучший)**   | 0.857        | 0.907          | 0.684  | 0.776  | 0.491     | `lr=0.014`, `depth=3`, `n_estim=863`, `scale_pos` |

> **Лучший результат достигнут моделью LightGBM:**  
> *ROC AUC = 0.907*, *PR AUC = 0.684*, что соответствует высокому охвату (Recall ≈ 0.78) при приемлемой для бизнеса точности (Precision ≈ 0.49).

---

### 2. Подробное описание и анализ экспериментов

#### 0. Baseline (Logistic Regression)
**Цель:** Создать отправную точку для сравнения. Была построена модель логистической регрессии. С помощью `GridSearchCV` оптимально подобраны сила (C) и тип (L1) регуляризации, а также стратегия балансировки классов.

Модель показала **ROC AUC = 0.870**, подтвердив наличие линейных зависимостей в данных. Высокий `Recall` (0.731) при низком `Precision` (0.408) говорит о том, что модель хорошо находит целевых клиентов, но ценой большого количества ложноположительных срабатываний — приемлемый компромисс для недорогих маркетинговых кампаний.

---

#### 1. Feature Engineering
*   **Цель:** Обогатить данные и проверить гипотезы о поведении клиентов. В датасет были добавлены новые признаки, отражающие бизнес-логику:
    *   *Мультиканальность*: `ChannelCount` и флаг `IsMultichannel`.
    *   *Агрегированные показатели*: `TotalMnt`, `TotalPurchases`, `AvgPurchaseValue`.
    *   *Социально-демографический флаг*: `IsParent`.

**Этот шаг дал самый значительный и "дешёвый" прирост качества**. Метрика **PR AUC**, наиболее чувствительная к улучшениям в условиях дисбаланса, выросла **с 0.573 до 0.663 (+15.7%)**. Это доказывает, что сгенерированные признаки несут в себе ценную бизнес-информацию, которую модель смогла успешно использовать для более точного разделения классов.

---

#### 2. Feature Selection
**Цель:** Удалить избыточные и "шумные" признаки для повышения стабильности модели. Применён двухэтапный отбор: сначала **корреляционный фильтр** удалил 3 признака с `|ρ| > 0.9`, затем **RFECV** (рекурсивное исключение признаков) оставил 23 наиболее сильных предиктора.

Отбор признаков привел к дальнейшему росту **ROC AUC до 0.889**. `Recall` значительно вырос до **0.836**: удаление "шума" позволило модели стать более чувствительной и увереннее находить представителей целевого класса, не теряя при этом в точности.

---

#### 3. Смена модели на ансамбли
**Цель:** Проверить, смогут ли более сложные нелинейные модели превзойти логистическую регрессию.

На лучшем наборе из 23 признаков были обучены и настроены три топовых ансамблевых алгоритма. Все три ансамбля значительно превзошли линейную модель, значит в данных есть нелинейные зависимости. `HistGradientBoosting` и `RandomForest` показали очень схожие и высокие результаты (ROC AUC ~0.90), существенно повысив `Precision` до ~0.53.

**LightGBM** оказался лидером, достигнув **ROC AUC 0.907** и **PR AUC 0.684**. Эта модель нашла наилучший баланс между охватом (Recall 0.776) и точностью.

---

### 3. Ключевые факторы роста качества

1.  **Инженерия признаков:** Добавление бизнес-метрик (`IsMultichannel`, `AvgPurchaseValue`) дало главный рывок в качестве (**+0.090 PR-AUC**).
2.  **Переход к градиентному бустингу:** Смена линейной модели на ансамблевую позволила учесть нелинейные зависимости и дала следующий по значимости прирост (**+0.018 ROC-AUC** к лучшему результату логистической регрссии).
3.  **Отбор признаков** улучшил `Recall`, сделав модель более надежной, хотя прирост по основным метрикам небольшой.

> **Итог:** Максимальное качество было достигнуто за счет синергии трех подходов: **продуманной инженерии признаков**, **тщательного отбора** и **использования алгоритма LightGBM**.

---

### 4. Важнейшие признаки в финальной модели (LightGBM, top-10 по gain)

1.  `NumWebVisitsMonth`
2.  `Recency`
3.  `ChannelCount`
4.  `NumCatalogPurchases`
5.  `Income`
6.  `TotalPurchases`
7.  `MntWines`
8.  `IsParent`
9.  `Education_PhD`
10. `AvgPurchaseValue`

**Анализ:** Сгенерированные нами признаки `ChannelCount`, `TotalPurchases`, `IsParent` и `AvgPurchaseValue` вошли в топ-10, подтвердив свою высокую предсказательную силу. В частности, `ChannelCount` в топ-3 доказывает, что гипотеза о важности мультиканальности была верна.


### 6. Заключение и рекомендации

Финальная модель на основе **LightGBM**, обученная на расширенном и очищенном наборе признаков, является оптимальным решением и гарантирует лучший баланс метрик:
*   **Высокий охват** (Recall ~ 78%), позволяющий не упустить большинство потенциальных клиентов.
*   **Приемлемую точность** (Precision ~ 49%), делающую кампанию экономически эффективной.
*   **Максимальные интегральные метрики** (ROC AUC = 0.907, PR AUC = 0.684), подтверждающие способность хорошо разделять покупателей на классы.

**Рекомендуется** использовать именно эту модель для таргетирования в будущей маркетинговой кампании. Признак мультиканальности клиента следует рассматривать как один из ключевых флагов для сегментации "тёплой" аудитории.